# Susceptibility Calculation Demo

This notebook demonstrates the susceptibility calculation module for quantum measurement simulations.

## Overview

Magnetic susceptibility (χ) measures the linear response of a quantum system to external perturbations:

$$\chi_{ij} = \int_0^T dt \, [\langle\sigma_i^z(t)\sigma_j^z(0)\rangle - \langle\sigma_i^z(t)\rangle\langle\sigma_j^z(0)\rangle]$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quantum_measurement.jw_expansion import TwoQubitCorrelationSimulator, LQubitCorrelationSimulator
from quantum_measurement.susceptibility import (
    compute_static_susceptibility,
    compute_connected_correlation,
    compute_susceptibility_matrix
)
from quantum_measurement.susceptibility.analytical import (
    analytical_susceptibility_from_decay_rate
)

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Example 1: Two-Qubit Susceptibility

Calculate susceptibility for a two-qubit system with XX coupling and continuous measurement.

In [ ]:
# Create two-qubit simulator
sim2 = TwoQubitCorrelationSimulator(
    J=1.0,          # Coupling strength
    epsilon=0.1,    # Measurement strength
    N_steps=1000,   # Number of time steps
    T=10.0          # Total time
)

print(f"Parameters:")
print(f"  J = {sim2.J}")
print(f"  ε = {sim2.epsilon}")
print(f"  N = {sim2.N_steps}")
print(f"  T = {sim2.T}")
print(f"  dt = {sim2.dt:.6f}")

In [ ]:
# Run a single trajectory
Q, z_trajectory, xi_trajectory = sim2.simulate_trajectory()

print(f"Entropy production: Q = {Q:.3f}")
print(f"Trajectory shape: {z_trajectory.shape}")

# Plot the trajectory
times = np.linspace(0, sim2.T, sim2.N_steps + 1)

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(times, z_trajectory[:, 0], label='⟨σ₁ᶻ⟩', alpha=0.7)
axes[0].plot(times, z_trajectory[:, 1], label='⟨σ₂ᶻ⟩', alpha=0.7)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('⟨σᶻ⟩')
axes[0].set_title('Magnetization Trajectory')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Compute and plot connected correlation
z1 = z_trajectory[:, 0]
connected_corr = compute_connected_correlation(z1, z1)

axes[1].plot(times[:len(connected_corr)], connected_corr, 'g-', linewidth=2)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Connected Correlation')
axes[1].set_title('C(t) = ⟨σ₁ᶻ(t)σ₁ᶻ(0)⟩ - ⟨σ₁ᶻ⟩²')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Compute susceptibilities
chi_00 = sim2.compute_susceptibility(z_trajectory, site_i=0, site_j=0)
chi_11 = sim2.compute_susceptibility(z_trajectory, site_i=1, site_j=1)
chi_01 = sim2.compute_susceptibility(z_trajectory, site_i=0, site_j=1)

print(f"\nSusceptibilities:")
print(f"  χ₀₀ = {chi_00:.4f}")
print(f"  χ₁₁ = {chi_11:.4f}")
print(f"  χ₀₁ = {chi_01:.4f}")

# Visualize as a matrix
chi_matrix = np.array([[chi_00, chi_01], [chi_01, chi_11]])

plt.figure(figsize=(6, 5))
im = plt.imshow(chi_matrix, cmap='RdBu_r', aspect='auto')
plt.colorbar(im, label='χ')
plt.xticks([0, 1], ['Site 0', 'Site 1'])
plt.yticks([0, 1], ['Site 0', 'Site 1'])
plt.title('Two-Qubit Susceptibility Matrix')

# Add values as text
for i in range(2):
    for j in range(2):
        text = plt.text(j, i, f'{chi_matrix[i, j]:.3f}',
                       ha="center", va="center", color="black")

plt.tight_layout()
plt.show()

## Example 2: Ensemble Averaging

Compute susceptibility statistics over multiple trajectories.

In [ ]:
# Simulate ensemble
n_trajectories = 50
print(f"Simulating {n_trajectories} trajectories...")

Q_values, z_trajectories, xi_trajectories = sim2.simulate_ensemble(
    n_trajectories=n_trajectories,
    progress=True
)

print(f"\nEntropy production statistics:")
print(f"  ⟨Q⟩ = {np.mean(Q_values):.3f} ± {np.std(Q_values):.3f}")

In [ ]:
# Compute susceptibility for each trajectory
chi_00_values = []
chi_01_values = []

for i in range(n_trajectories):
    chi_00 = sim2.compute_susceptibility(z_trajectories[i], site_i=0, site_j=0)
    chi_01 = sim2.compute_susceptibility(z_trajectories[i], site_i=0, site_j=1)
    chi_00_values.append(chi_00)
    chi_01_values.append(chi_01)

chi_00_values = np.array(chi_00_values)
chi_01_values = np.array(chi_01_values)

print(f"\nSusceptibility statistics:")
print(f"  ⟨χ₀₀⟩ = {np.mean(chi_00_values):.4f} ± {np.std(chi_00_values):.4f}")
print(f"  ⟨χ₀₁⟩ = {np.mean(chi_01_values):.4f} ± {np.std(chi_01_values):.4f}")

# Plot histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(chi_00_values, bins=20, alpha=0.7, edgecolor='black')
axes[0].axvline(np.mean(chi_00_values), color='r', linestyle='--', 
                linewidth=2, label=f'Mean = {np.mean(chi_00_values):.4f}')
axes[0].set_xlabel('χ₀₀')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of χ₀₀')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(chi_01_values, bins=20, alpha=0.7, edgecolor='black', color='orange')
axes[1].axvline(np.mean(chi_01_values), color='r', linestyle='--',
                linewidth=2, label=f'Mean = {np.mean(chi_01_values):.4f}')
axes[1].set_xlabel('χ₀₁')
axes[1].set_ylabel('Count')
axes[1].set_title('Distribution of χ₀₁')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 3: L-Qubit Chain Susceptibility

Calculate the full susceptibility matrix for a multi-qubit chain.

In [ ]:
# Create L-qubit simulator
L = 5
simL = LQubitCorrelationSimulator(
    L=L,
    J=1.0,
    epsilon=0.1,
    N_steps=1000,
    T=10.0,
    closed_boundary=False  # Open boundary conditions
)

print(f"L-qubit chain with L={L}")
print(f"Boundary conditions: Open")
print(f"J = {simL.J}, ε = {simL.epsilon}")

In [ ]:
# Run trajectory
Q, z_trajectory, xi_trajectory = simL.simulate_trajectory()

print(f"Entropy production: Q = {Q:.3f}")

# Plot magnetization for all sites
times = np.linspace(0, simL.T, simL.N_steps + 1)

plt.figure(figsize=(12, 6))
for i in range(L):
    plt.plot(times, z_trajectory[:, i], label=f'Site {i}', alpha=0.7)

plt.xlabel('Time')
plt.ylabel('⟨σᶻ⟩')
plt.title(f'{L}-Qubit Chain Magnetization Dynamics')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Compute full susceptibility matrix
chi_matrix = simL.compute_susceptibility_matrix(z_trajectory)

print(f"\nSusceptibility matrix ({L}×{L}):")
print(chi_matrix)
print(f"\nDiagonal elements: {np.diag(chi_matrix)}")

# Visualize the susceptibility matrix
plt.figure(figsize=(8, 6))
im = plt.imshow(chi_matrix, cmap='RdBu_r', aspect='auto')
plt.colorbar(im, label='χᵢⱼ')
plt.xlabel('Site j')
plt.ylabel('Site i')
plt.title(f'{L}-Qubit Susceptibility Matrix')
plt.xticks(range(L))
plt.yticks(range(L))

# Add grid
for i in range(L+1):
    plt.axhline(i-0.5, color='gray', linewidth=0.5, alpha=0.3)
    plt.axvline(i-0.5, color='gray', linewidth=0.5, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Study spatial dependence: χ(r) vs distance
site_0 = 0  # Reference site
chi_vs_distance = []

for r in range(L):
    chi_r = simL.compute_susceptibility(z_trajectory, site_i=site_0, site_j=r)
    chi_vs_distance.append(chi_r)

plt.figure(figsize=(10, 5))
plt.plot(range(L), chi_vs_distance, 'o-', markersize=8, linewidth=2)
plt.xlabel('Distance r from site 0')
plt.ylabel(f'χ₀ᵣ')
plt.title('Susceptibility vs Spatial Separation')
plt.grid(True, alpha=0.3)
plt.xticks(range(L))
plt.tight_layout()
plt.show()

print("\nSusceptibility vs distance:")
for r, chi in enumerate(chi_vs_distance):
    print(f"  χ(r={r}) = {chi:.4f}")

## Example 4: Dependence on Measurement Strength

Study how susceptibility changes with measurement strength ε.

In [ ]:
# Scan over different measurement strengths
epsilon_values = [0.05, 0.1, 0.15, 0.2, 0.3]
chi_vs_epsilon = []

print("Computing susceptibility for different ε...")
for eps in epsilon_values:
    sim = TwoQubitCorrelationSimulator(J=1.0, epsilon=eps, N_steps=1000, T=10.0)
    Q, z_traj, _ = sim.simulate_trajectory()
    chi = sim.compute_susceptibility(z_traj, site_i=0)
    chi_vs_epsilon.append(chi)
    print(f"  ε = {eps:.2f}: χ = {chi:.4f}")

# Analytical prediction: χ ∝ 1/ε² (for strong measurement limit)
analytical_chi = [analytical_susceptibility_from_decay_rate(eps**2) for eps in epsilon_values]

plt.figure(figsize=(10, 6))
plt.plot(epsilon_values, chi_vs_epsilon, 'o-', markersize=8, linewidth=2, 
         label='Numerical', color='blue')
plt.plot(epsilon_values, analytical_chi, '--', linewidth=2, 
         label='Analytical (1/ε²)', color='red', alpha=0.7)
plt.xlabel('Measurement strength ε')
plt.ylabel('Susceptibility χ')
plt.title('Susceptibility vs Measurement Strength')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Two-qubit susceptibility** - Computing χ for individual sites and cross-susceptibility
2. **Ensemble averaging** - Statistical analysis over multiple trajectories
3. **L-qubit chains** - Full susceptibility matrices and spatial correlations
4. **Parameter dependence** - How susceptibility varies with measurement strength

Key observations:
- Susceptibility quantifies correlation strength and decay rate
- Stronger measurement (larger ε) → smaller χ (faster correlation decay)
- Spatial correlations decay with distance in open chains
- Connected correlations isolate genuine quantum correlations